In [1]:
#import streamlit as st

import os, json
import openai
import re
from langchain.chat_models import AzureChatOpenAI
from langchain.llms import AzureOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains.question_answering import load_qa_chain
from langchain.schema import BaseRetriever
from langchain.vectorstores.azuresearch import AzureSearch
from langchain.vectorstores import FAISS
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from azure.storage.blob import BlobServiceClient

In [2]:
# !pip install --upgrade langchain
# !pip install azure-storage-blob

In [3]:
openai.api_type = "azure"
#openai.api_base = ""
openai.api_base = ""

#openai.api_version = "2022-12-01"
openai.api_version = "2023-03-15-preview"

#https://openai-oncology-bot.openai.azure.com/openai/deployments/gpt-35-turbo/chat/completions?api-version=2023-03-15-preview

os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.getenv("OPENAI_API_KEY")
#openai.api_version = "2022-12-01"

In [4]:
# vector_store_address = ""
# vector_store_password: str = ""
#index_name: str = "langchain-vector-demo"
index_name: str = "random-pdf-index"    
vector_store_address = ""
vector_store_password: str = ""

In [5]:
import requests
#API_KEY = ""
API_KEY = ""

RESOURCE_ENDPOINT = ""

openai.api_type = "azure"
openai.api_key = API_KEY
openai.api_base = RESOURCE_ENDPOINT
openai.api_version = "2023-03-15-preview"

url = openai.api_base + "/openai/deployments?api-version=2023-03-15-preview"
#?api-version=2023-03-15-preview"

r = requests.get(url, headers={"api-key": API_KEY})

 
#l
 

#print(r.text)

In [6]:
llm = AzureOpenAI(deployment_name="gpt-35-turbo",model_name="gpt-35-turbo",temperature = 0, openai_api_version = "2023-03-15-preview" )
embeddings = OpenAIEmbeddings(deployment= "Text-embedding-ada-002", model="text-embedding-ada-002", chunk_size=1)
model = "text-embedding-ada-002" 

In [7]:
import os

In [8]:
os.listdir()

['.ipynb_checkpoints',
 'ACS.ipynb',
 'config.py',
 'data',
 'index',
 'Oncology',
 'Oncology.rar',
 'Oncology_index',
 'Saving_index_to_blob (1).ipynb',
 'script_to_answer_query (1).ipynb',
 'Untitled.ipynb',
 '__pycache__']

<h4> load and chunk the data for embedding

In [12]:
%%time
directory = 'data'

def load_docs(directory):
    loader = DirectoryLoader(directory)
    documents = loader.load()
    return documents

documents = load_docs(directory)


CPU times: total: 3min 2s
Wall time: 3min 2s


In [13]:
%%time
def text_splitter(documents):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=30)
    docs = text_splitter.split_documents(documents)
    return docs

docs = text_splitter(documents)

CPU times: total: 234 ms
Wall time: 225 ms


In [14]:
docs

[Document(page_content='1149577 JMCXXX10.1177/10776958221149577Journalism & Mass Communication EducatorPavlik\n\nresearch-article2023\n\nEssay\n\nJournalism & Mass Communication Educator 2023, Vol. 78(1) 84 –93 © AEJMC 2023 Article reuse guidelines: sagepub.com/journals-permissions DOI: 10.1177/10776958221149577 https://doi.org/10.1177/10776958221149577 http://journals.sagepub.com/home/jmc\n\nCollaborating With ChatGPT: Considering the Implications of Generative Artificial Intelligence for Journalism and Media Education\n\nJohn V. Pavlik1', metadata={'source': 'data\\Collaborating With GPT.pdf'}),
 Document(page_content='John V. Pavlik1\n\nAbstract Generative artificial intelligence (AI) is ushering in an era of potential transformation of journalism and media content. This essay considers one notable generative AI platform called ChatGPT made available to the public in 2022 for free use. ChatGPT allows users to enter text prompts and rapidly generates text responses drawn from its kno

In [ ]:
len(docs)

In [ ]:
# !pip install --index-url=https://pkgs.dev.azure.com/azure-sdk/public/_packaging/azure-sdk-for-python/pypi/simple/ azure-search-documents==11.4.0a20230509004
# !pip install azure-identity

<h4> ACS embedding

In [15]:
embeddings: OpenAIEmbeddings = OpenAIEmbeddings(model=model, chunk_size=1)
vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint=vector_store_address,
    azure_search_key=vector_store_password,
    index_name=index_name,
    embedding_function=embeddings.embed_query,
)

In [16]:
%%time
vector_store.add_documents(documents=docs)

CPU times: total: 27.9 s
Wall time: 17min 50s


['...',
 ...]

In [17]:
len(docs)

3768

<h3>Download FAISS embedding from blob if not already stored in local

In [ ]:
storage_connection_string = 'DefaultEndpointsProtocol=https;AccountName=storageoncology;AccountKey=0lVPmULxydJBSS0iqShbEjlFssRmRNn6cx4XLPdCM/CMvm6+YsQI5Lc2xMz12EoOY3GKIjcDjZNE+AStaN5+Ug==;EndpointSuffix=core.windows.net'

In [ ]:
def download_blob_folder(storage_connection_string, container_name, folder_name, destination_path):
    try:
        os.makedirs(destination_path, exist_ok=True)  # Create the destination folder if it doesn't exist
    except:
        pass

    blob_service_client = BlobServiceClient.from_connection_string(storage_connection_string)
    container_client = blob_service_client.get_container_client(container_name)

    # List blobs within the folder
    blob_list = container_client.list_blobs(name_starts_with=folder_name)

    for blob in blob_list:
        # Construct the destination path by removing the folder name from the blob name
        destination_blob_name = blob.name[len(folder_name):].lstrip('/')

        # Download the blob to the specified destination
        blob_client = container_client.get_blob_client(blob.name)
        with open(os.path.join(destination_path, destination_blob_name), 'wb') as file:
            file.write(blob_client.download_blob().readall())

# Provide your storage connection string, container name, folder name, and local destination path
storage_connection_string = storage_connection_string
container_name = "fais-embed"
folder_name = "index1"
destination_path = "Oncology_index1"

download_blob_folder(storage_connection_string, container_name, folder_name, destination_path)

<h4> FAISS vector embedding

In [ ]:
folder_name = "Oncology_index1"

destination_path = folder_name

In [ ]:
#Generate FAISS embedding
def get_embedings(docs,embeddings, folder_name):
    embeddings =  embeddings
    docsearch = FAISS.from_documents(docs, embeddings)
    os.makedirs(folder_name, exist_ok=True)
    docsearch.save_local(folder_name)
    
    
get_embedings(docs, embeddings, folder_name)


In [ ]:
#load the embedding once generated or downloaded from blob
db = FAISS.load_local(destination_path, embeddings)

In [ ]:
def acsContext(query):
    docs = vector_store.similarity_search(
    query=query,
    k=3,
    search_type="similarity",)  
    return docs[0].page_content

In [ ]:
def acsResponse(query):
    docs = vector_store.similarity_search(
    query=query,
    k=3,
    search_type="similarity",)  
    
    # Build the prompt
    prompt = f"""
    Answer the following question based on the context below.
    If you don't know the answer, just say that you don't know. Don't try to make up an answer. Do not answer beyond this context.
    ---
    QUESTION: {query}                                            
    ---
    CONTEXT:
    {docs}
    """
    # Run chat completion using GPT-4
    response = openai.ChatCompletion.create(
    engine="gpt-35-turbo",
    messages=[
        { "role": "system", "content":  "You are a Q&A assistant." },
        { "role": "user", "content": prompt }
    ],
    temperature=0.2,
    max_tokens=1000
    )


    return (response.choices[0]['message']['content'])

    
    

In [ ]:
def FaissContext(query):
    docs = db.similarity_search(query)
    return docs[0].page_content

In [ ]:
def FaissResponse(query):
    docs = db.similarity_search(query) 
    
    # Build the prompt
    prompt = f"""
    Answer the following question based on the context below.
    If you don't know the answer, just say that you don't know. Don't try to make up an answer. Do not answer beyond this context.
    ---
    QUESTION: {query}                                            
    ---
    CONTEXT:
    {docs}
    """
    # Run chat completion using GPT-4
    response = openai.ChatCompletion.create(
    engine="gpt-35-turbo",
    messages=[
        { "role": "system", "content":  "You are a Q&A assistant." },
        { "role": "user", "content": prompt }
    ],
    temperature=0.2,
    max_tokens=1000
    )


    return (response.choices[0]['message']['content'])

    
    
    

In [ ]:
import pandas as pd


# Specify the path to your Excel file
# excel_file_path = "https://celebaltech-my.sharepoint.com/:x:/p/lokendra_singh/ERFlmIwcDF5NvYTNQyQPUv4B0bUH7C8g6LhGZIklpXUlhQ?e=yZVoRo"
excel_file_path = "pdf questions.xlsx"
# Read the Excel file into a pandas DataFrame
df = pd.read_excel(excel_file_path)

print(df.columns)

In [ ]:
# Find the column index for "questions" and "answers"
# questions_column = "Questions"
# answers_column = "ACS GPT answer"

# Iterate over the rows and process the questions
for index, row in df.iterrows():
    question = row['Questions']

    # Process question 1
#     answer_1 = acsContext(question)
#     df.at[index, 'ACS GPT answer '] = answer_1

    # Process question 2
#     answer_2 = acsResponse(question)
#     df.at[index, 'ACS context'] = answer_2

    # Process question 3
    answer_3 = FaissContext(question)
    df.at[index, 'FAISS GPT answer'] = answer_3
    
    
    
    answer_4 = FaissResponse(question)
    df.at[index, 'FAISS context'] = answer_4

# Save the modified DataFrame back to the Excel file
df.to_excel(excel_file_path, index=False)


In [ ]:
# # Find the column indices for "Questions" and "ACS GPT answer"
# questions_column = df.columns.get_loc("Questions")
# #answers_column = df.columns.get_loc("ACS GPT answer")

# # Iterate over the rows and process the questions
# for index, row in df.iterrows():
#     question = row[questions_column]

#     # Process question 1
#     answer_1 = acsContext(question)
#     df.loc[index, 'acsContext'] = answer_1

#     # Process question 2
#     answer_2 = acsResponse(question)
#     df.loc[index, 'acsResponse'] = answer_2

#     # Process question 3
#     answer_3 = FaissContext(question)
#     df.loc[index, 'FaissContext'] = answer_3
    
#     answer_4 = FaissResponse(question)
#     df.loc[index, 'FaissResponse'] = answer_4

# # Save the modified DataFrame back to the Excel file
# df.to_excel(excel_file_path, index=False)


In [74]:
query = "What are laws for OTT in different countries?"

In [79]:
# docs = db.similarity_search(query)
# #print(docs[0].page_content)
# print(docs)

In [80]:
docs = vector_store.similarity_search(
query=query,
k=3,
search_type="similarity",)  
#print(docs[0].page_content)
# print(docs)

In [81]:
# Build the prompt
prompt = f"""
Answer the following question based on the context below.
If you don't know the answer, just say that you don't know. Don't try to make up an answer. Do not answer beyond this context.
---
QUESTION: {query}                                            
---
CONTEXT:
{docs}
"""


In [82]:
# Run chat completion using GPT-4
response = openai.ChatCompletion.create(
    #deployment_id = "text-davinci-003",
    engine="gpt-35-turbo",
    messages=[
        { "role": "system", "content":  "You are a Q&A assistant." },
        { "role": "user", "content": prompt }
    ],
    temperature=0.0,
    max_tokens=1000
)


print(response.choices[0]['message']['content'])


Based on the given context, it is mentioned that online video subscription sales, including subscription video-on-demand (SVOD) and transactional video-on-demand (TVOD) services, are predicted to rise from $12.5 billion in 2014 to $45.26 billion in 2019. However, there is no specific information provided about the online video subscription sale of video of Ott between 2014 to 2019.
